In [1]:
import os 
import pandas as pd

In [2]:
#Import data
path ="../Extract/Time_Series_19-covid-Confirmed Cases.csv"

covid_original = pd.read_csv(path)
covid_original.head()

,Province/State,Country/Region,Lat,Long,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,...,2/26/2020,2/27/2020,2/28/2020,2/29/2020,3/1/2020,3/2/2020,3/3/2020,3/4/2020,3/5/2020,3/6/2020
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,990,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,400,410,410,411,413,414,414,418,418,422
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,294,296,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,102,119


In [3]:
#Import data
path ="../Extract/Country_Weather.csv"

weather_original = pd.read_csv(path)
weather_original.head()

,Country,Latitude,Longitude,Elevation,Date,Precipitation,Temp Avg,Temp Max,Temp Min
0,China,30.6,114.05,34.0,1/22/2020,0.48,42.0,46.0,39.0
1,China,30.6,114.05,34.0,1/23/2020,0.13,45.0,NaN,39.0
2,China,30.6,114.05,34.0,1/24/2020,0.23,43.0,47.0,NaN
3,China,30.6,114.05,34.0,1/25/2020,0.18,40.0,45.0,37.0
4,China,30.6,114.05,34.0,1/26/2020,0.02,38.0,42.0,NaN


In [4]:
# Drop columns
covid19_df = covid_original.drop(columns=['Province/State'])

# Pull dates out of column head and into df
covid19_df1 = covid19_df.set_index(['Country/Region','Lat','Long']).stack().to_frame()

# Reset index and rename column
covid19_df1.reset_index(inplace=True)
covid19_df1.rename(columns={'Country/Region':'Country', 'level_3':'Date', 0:'Confirmed Cases'}, inplace=True)
covid19_df1

,Country,Lat,Long,Date,Confirmed Cases
0,Mainland China,31.8257,117.2264,1/22/2020,1
1,Mainland China,31.8257,117.2264,1/23/2020,9
2,Mainland China,31.8257,117.2264,1/24/2020,15
3,Mainland China,31.8257,117.2264,1/25/2020,39
4,Mainland China,31.8257,117.2264,1/26/2020,60
...,...,...,...,...,...
8950,Vatican City,41.9029,12.4534,3/2/2020,0
8951,Vatican City,41.9029,12.4534,3/3/2020,0
8952,Vatican City,41.9029,12.4534,3/4/2020,0
8953,Vatican City,41.9029,12.4534,3/5/2020,0


In [5]:
# Grouping cases by country
covid19_df2 =covid19_df1.groupby(['Country', 'Date']).agg({'Lat':'mean', 'Long':'mean','Confirmed Cases':'sum'})
covid19_df2.reset_index(inplace=True)

# Renaming Country
covid19_df2.replace('Mainland China', 'China', inplace=True)

# Sort Countries alpabetically 
covid19_df2.sort_values(by= ['Country', 'Date'], inplace=True)

covid19_df2

,Country,Date,Lat,Long,Confirmed Cases
0,Afghanistan,1/22/2020,33.0,65.0,0
1,Afghanistan,1/23/2020,33.0,65.0,0
2,Afghanistan,1/24/2020,33.0,65.0,0
3,Afghanistan,1/25/2020,33.0,65.0,0
4,Afghanistan,1/26/2020,33.0,65.0,0
...,...,...,...,...,...
4450,Vietnam,3/2/2020,16.0,108.0,16
4451,Vietnam,3/3/2020,16.0,108.0,16
4452,Vietnam,3/4/2020,16.0,108.0,16
4453,Vietnam,3/5/2020,16.0,108.0,16


In [6]:
# Creating a list of all the countries and turning it into a df
country_list = covid19_df2['Country'].unique().tolist()
country_df1 = pd.DataFrame(country_list)
country_df = country_df1.reset_index()
country_df = country_df.rename(columns = {0:'Country'})
country_df

,index,Country
0,0,Afghanistan
1,1,Algeria
2,2,Andorra
3,3,Argentina
4,4,Armenia
...,...,...
94,94,US
95,95,Ukraine
96,96,United Arab Emirates
97,97,Vatican City


In [7]:
#Save df to a csv file
country_df.to_csv ('country_list_updated.csv', index = False, header = True)

In [8]:
# Merge country key into Covid 19 df
covid19_key_df = pd.merge(country_df, covid19_df2, on="Country")

# Renmaing columns and resetting index
covid19_key_df1 = covid19_key_df.rename(columns = {'index':' Country Key'})
covid19_cleaned_df = covid19_key_df1.reset_index()
covid19_cleaned_df

,index,Country Key,Country,Date,Lat,Long,Confirmed Cases
0,0,0,Afghanistan,1/22/2020,33.0,65.0,0
1,1,0,Afghanistan,1/23/2020,33.0,65.0,0
2,2,0,Afghanistan,1/24/2020,33.0,65.0,0
3,3,0,Afghanistan,1/25/2020,33.0,65.0,0
4,4,0,Afghanistan,1/26/2020,33.0,65.0,0
...,...,...,...,...,...,...,...
4450,4450,98,Vietnam,3/2/2020,16.0,108.0,16
4451,4451,98,Vietnam,3/3/2020,16.0,108.0,16
4452,4452,98,Vietnam,3/4/2020,16.0,108.0,16
4453,4453,98,Vietnam,3/5/2020,16.0,108.0,16


In [9]:
# Save df to a csv file
covid19_cleaned_df.to_csv('covid19_updated.csv', index = False, header = True)

In [10]:
# Drop unneeded columns
weather_df = weather_original[['Country', 'Date', 'Precipitation', 'Temp Avg', 'Temp Max', 'Temp Min']]

# Renaming Countries
weather_df.replace('Bosnia', 'Bosnia and Herzegovina', inplace=True)

# Merge country key with this df
weather_key_df = pd.merge(country_df, weather_df, on="Country")

# Reset Index to make primary key, rename columns and sort values
weather_key_df.reset_index(inplace=True)
weather_cleaned_df = weather_key_df.rename(columns={'level_0':'index', 'index': 'Country Key'})
weather_cleaned_df.sort_values(by= ['Country', 'Date'], inplace=True)
weather_cleaned_df

/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,index,Country Key,Country,Date,Precipitation,Temp Avg,Temp Max,Temp Min
0,0,0,Afghanistan,1/22/2020,NaN,43.0,55.0,34.0
1,1,0,Afghanistan,1/23/2020,NaN,42.0,56.0,33.0
2,2,0,Afghanistan,1/24/2020,0.17,40.0,43.0,37.0
3,3,0,Afghanistan,1/25/2020,0.57,46.0,56.0,38.0
4,4,0,Afghanistan,1/26/2020,0.00,43.0,53.0,36.0
...,...,...,...,...,...,...,...,...
3499,3499,98,Vietnam,2/7/2020,NaN,78.0,89.0,NaN
3500,3500,98,Vietnam,2/8/2020,0.09,80.0,90.0,NaN
3501,3501,98,Vietnam,2/9/2020,0.00,81.0,NaN,NaN
3522,3522,98,Vietnam,3/1/2020,NaN,81.0,NaN,NaN


In [11]:
# Save df to a csv file
weather_cleaned_df.to_csv('weather_updated.csv', index = False, header = True)